IMPORTAÇÃO DE MODULOS

In [1]:
from path_arquivos import *
import pandas as pd
import numpy as np

LEITURA DOS DATAFRAMES

In [2]:
df_prod = pd.read_excel(ar_xlsx.ar_96, usecols= ['CODPROD', 'QTUNITCX', 'CAPACIDADE', 'PONTOREPOSICAO','QTTOTPAL'])
df_bloq = pd.read_excel(ar_xls.ar_86, usecols=['Código', 'Bloqueado(Qt.Bloq.-Qt.Avaria)'])
end_ger = pd.read_csv(ar_csv.ar_07, header= None, names= col_name.c07)  
df_end = pd.read_csv(ar_csv.ar_end, header= None, names= col_name.cEnd)



c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


CORPO DO SCRIPT

In [3]:
dic_end_ger = {'COD' : "CODPROD"}
end_ger = end_ger.rename(columns= dic_end_ger)
end_ger['RUA'] = end_ger['RUA'].fillna(0).astype(int)

dic_bloq = {'Código' : "CODPROD", 'Bloqueado(Qt.Bloq.-Qt.Avaria)' : "BLOQ"}
df_bloq = df_bloq.rename(columns= dic_bloq)
df_bloq['CODPROD'] = df_bloq['CODPROD'].fillna(0).astype(int)

dic_end = {"COD" : "CODPROD"}
df_end = df_end.rename(columns= dic_end)
df_end = df_end[['CODPROD','ENTRADA', 'SAIDA', 'DISP','QTDE']]
col_ajuste = ['CODPROD','ENTRADA', 'SAIDA', 'DISP','QTDE']
for col in col_ajuste:
    df_end[col] = df_end[col].fillna(0).astype(str)
    df_end[col] = df_end[col].str.replace(".", "")
    df_end[col] = df_end[col].str.replace(",", ".")
    df_end[col] = df_end[col].fillna(0).astype(float)
grupo_end = df_end.groupby('CODPROD').agg(
    SAIDA = ('SAIDA', 'sum'),
    ENTRADA = ('ENTRADA', 'sum'),
    DISP = ('DISP', 'sum'),
    QTDE = ('QTDE', 'sum'),
).reset_index()

df_prod['CODPROD'] = df_prod['CODPROD'].fillna(0).astype(int)

df = end_ger.loc[end_ger['RUA'].between(1, 39)]
df = df.merge(df_bloq, left_on='CODPROD',right_on='CODPROD', how= 'left')
df = df.merge(grupo_end, left_on='CODPROD',right_on='CODPROD', how= 'left')
df = df.merge(df_prod, left_on='CODPROD',right_on='CODPROD', how= 'left')


VISUALIZAÇÃO

In [7]:
df[['RUA', 'PREDIO']] = df[['RUA', 'PREDIO']].astype(int)
df = df.sort_values(by=['RUA', 'PREDIO'], ascending= True)
display(df.head())

,CODPROD,DESCRICAO,DEP,RUA,PREDIO,NIVEL,APTO,EMBALAGEM,CAP,P_REP,...,GERENCIAL,BLOQ,SAIDA,ENTRADA,DISP,QTDE,QTUNITCX,QTTOTPAL,CAPACIDADE,PONTOREPOSICAO
3059,451985,CAIXA TERMICA TERMOLAR 32L VM 3502,1.0,1,3,1.0,101.0,CX/0001/UN,40.0,20.0,...,"144,00",0.0,0.0,0.0,120.0,120.0,1,20,40,20
1067,430734,UD GARRAFAO T TERMOLAR 5L AZ 1095,1.0,1,5,1.0,101.0,UN/0001/UN,40.0,20.0,...,"968,00",0.0,0.0,0.0,840.0,840.0,6,20,40,20
3058,451984,CAIXA TERMICA TERMOLAR 32L CZ 3502,1.0,1,7,1.0,101.0,CX/0001/UN,40.0,20.0,...,"112,00",NaN,0.0,0.0,80.0,80.0,1,20,40,20
1882,443712,UD COPO VDO AMERICANO NADIR 190ML M USO,1.0,1,8,1.0,101.0,CX/0024/UN,312.0,156.0,...,"4.947,00",0.0,0.0,0.0,4623.0,4623.0,1,156,312,156
7272,470423,UD VENTILADOR MALLORY 30CM TURB 220V,1.0,1,10,1.0,102.0,UN/0001/UN,64.0,16.0,...,"44,00",0.0,NaN,NaN,NaN,NaN,1,48,64,16
